$$
\begin{aligned}
\frac{S(t)}{dt} &= -\frac{\beta \cdot S(t) \cdot I(t)}{N} - \langle \delta \rangle \cdot S(t) \cdot E(t) \\
\frac{E(t)}{dt} &= \frac{\beta \cdot S(t) \cdot I(t)}{N} - \langle \alpha \rangle \cdot E(t) + \langle \delta \rangle \cdot S(t) \cdot E(t) \\
\frac{I(t)}{dt} &= \langle \alpha \rangle \cdot E(t) - \langle \gamma \rangle \cdot I(t) - \mu \cdot I(t) \\
\frac{R(t)}{dt} &= \langle \gamma \rangle \cdot I(t) \\
\frac{D(t)}{dt} &= \langle \mu \rangle \cdot I(t) \\ \\
\beta &= 0.5 \\
N &= S(t) + E(t) + I(t) + R(t) + D(t)
\end{aligned}
$$
